In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset
from nltk.translate.bleu_score import sentence_bleu
import torch

# Set device to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

In [3]:
def read_file_and_split(filename):
    kazakh_corpus = []
    eng_corpus = []

    try:
        with open(filename, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:  # Ensure there are exactly 3 parts: 2 sentences and a number
                    kazakh_sentence = parts[0].strip()
                    eng_sentence = parts[1].strip()
                    kazakh_corpus.append(kazakh_sentence)
                    eng_corpus.append(eng_sentence)
                else:
                    print("Invalid format in line:", line)

    except FileNotFoundError:
        print("File not found:", filename)

    return kazakh_corpus, eng_corpus

kazakh_corpus, eng_corpus = read_file_and_split("kazakh_eng_corpus.txt")

In [ ]:
# Initialize translator pipeline
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng_Latn", tgt_lang="kaz_Cyrl", batch_size=64, device=device)
test_data = eng_corpus

# Tokenize reference sentences
reference_sentences = [[tokenizer.tokenize(sentence)] for sentence in kazakh_corpus]
print(reference_sentences[0])

[['▁соңғы', '▁бес', '▁жылда', '▁ана', '▁ө', 'лімі', '▁шама', 'мен', '▁3', '▁есе', '▁аза', 'йды', '▁,', '▁бала', '▁туу', '▁көрсеткі', 'ші', '▁бір', '▁жа', 'рым', '▁есе', '▁ө', 'сті', '▁.']]


In [ ]:
total_bleu = 0.0
num_samples = len(test_data)
for i in range(num_samples):
  translated_sentence = translator(test_data[i])[0]['translation_text']
  translated_tokens = tokenizer.tokenize(translated_sentence)
  bleu_score = sentence_bleu(reference_sentences[i], translated_tokens)
  print("bleu score for sentence ", i, " is ", bleu_score)
  total_bleu += bleu_score

average_bleu = total_bleu / num_samples
print("Average bleu score is ", average_bleu)

bleu score for sentence  0  is  0.25805768448215954


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  1  is  6.961056329921462e-156
bleu score for sentence  2  is  0.005816598678330799


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  3  is  1.0385454635267381e-232
bleu score for sentence  4  is  4.2348350024066726e-156
bleu score for sentence  5  is  4.65988169467104e-155
bleu score for sentence  6  is  8.039302753516994e-80
bleu score for sentence  7  is  5.968436978771259e-156
bleu score for sentence  8  is  2.0171072364516604e-232
bleu score for sentence  9  is  8.726094729337945e-232


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


bleu score for sentence  10  is  4.1525423801808625e-233
bleu score for sentence  11  is  5.8813156969232236e-232
bleu score for sentence  12  is  0
bleu score for sentence  13  is  2.225692709577079e-232
bleu score for sentence  14  is  4.83188142069349e-233
bleu score for sentence  15  is  0
bleu score for sentence  16  is  2.060046999007818e-232
bleu score for sentence  17  is  8.231055179516831e-232
bleu score for sentence  18  is  1.5339208059700235e-155
bleu score for sentence  19  is  0
bleu score for sentence  20  is  0.02121007565527424
bleu score for sentence  21  is  2.1266336439355304e-155
bleu score for sentence  22  is  0
bleu score for sentence  23  is  7.659859945595747e-232
bleu score for sentence  24  is  0
bleu score for sentence  25  is  0.04879115949880177
bleu score for sentence  26  is  0
bleu score for sentence  27  is  8.231055179516831e-232
bleu score for sentence  28  is  1.2868286781332451e-155
bleu score for sentence  29  is  3.418291552750845e-232
bleu sco

In [ ]:
from datasets import load_dataset
flores_dataset = load_dataset("facebook/flores", "eng_Latn-kaz_Cyrl")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for facebook/flores contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/facebook/flores
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from nltk.translate.bleu_score import sentence_bleu
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("amandyk/mt5-kazakh-english-translation")
tokenizer = AutoTokenizer.from_pretrained("amandyk/mt5-kazakh-english-translation")

config.json:   0%|          | 0.00/798 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/475 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="kaz", tgt_lang="eng", batch_size=64, device=device, max_length=400)

In [ ]:
translator("Қалайсын?")

[{'translation_text': 'How to?'}]

In [ ]:
test_data = flores_dataset["devtest"]
reference_sentences = [[tokenizer.tokenize(sentence)] for sentence in test_data['sentence_eng_Latn']]

In [ ]:
total_bleu = 0.0
num_samples = len(test_data)
for i in range(num_samples):
  translated_sentence = translator(test_data['sentence_kaz_Cyrl'][i])[0]['translation_text']
  translated_tokens = tokenizer.tokenize(translated_sentence)
  bleu_score = sentence_bleu(reference_sentences[i], translated_tokens)
  print("bleu score for sentence ", i, " is ", bleu_score)
  total_bleu += bleu_score

average_bleu = total_bleu / num_samples
print("Average bleu score is ", average_bleu)

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

bleu score for sentence  0  is  1.2952538696889137e-231
bleu score for sentence  1  is  0.06875391242113171
bleu score for sentence  2  is  1.5332777658114095e-78
bleu score for sentence  3  is  0.06259075114179864
bleu score for sentence  4  is  2.1537116717116018e-78
bleu score for sentence  5  is  2.1384370837380596e-78
bleu score for sentence  6  is  7.55874475436192e-155
bleu score for sentence  7  is  4.400770623522883e-155
bleu score for sentence  8  is  8.726094729337945e-232
bleu score for sentence  9  is  4.2538705324613135e-155


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


bleu score for sentence  10  is  0.08982809212653368
bleu score for sentence  11  is  0.2316300906046136
bleu score for sentence  12  is  0.24664113540497917
bleu score for sentence  13  is  0.15450658933711403
bleu score for sentence  14  is  0.1670965345309382
bleu score for sentence  15  is  1.5670687366251056e-78
bleu score for sentence  16  is  0.2403154795553185
bleu score for sentence  17  is  5.001964139675266e-155
bleu score for sentence  18  is  0.2748619655125453
bleu score for sentence  19  is  2.863390591721491e-78
bleu score for sentence  20  is  8.120902309991712e-232
bleu score for sentence  21  is  1.3094850321578048e-231
bleu score for sentence  22  is  2.9137023823568156e-155
bleu score for sentence  23  is  0.34160103206640785
bleu score for sentence  24  is  2.3461804549581374e-78
bleu score for sentence  25  is  0.31902049719460357
bleu score for sentence  26  is  5.8726437213241354e-155
bleu score for sentence  27  is  1.836675436475691e-78
bleu score for sentenc

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="kk_KZ", tgt_lang="en_XX", batch_size=64, device=device, max_length=400)

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

In [ ]:
total_bleu = 0.0
num_samples = len(test_data)
for i in range(num_samples):
  translated_sentence = translator(test_data['sentence_kaz_Cyrl'][i])[0]['translation_text']
  translated_tokens = tokenizer.tokenize(translated_sentence)
  bleu_score = sentence_bleu(reference_sentences[i], translated_tokens)
  print("bleu score for sentence ", i, " is ", bleu_score)
  total_bleu += bleu_score

average_bleu = total_bleu / num_samples
print("Average bleu score is ", average_bleu)

bleu score for sentence  0  is  9.510170371400236e-232
bleu score for sentence  1  is  0.4546913155179998
bleu score for sentence  2  is  0.11571074323665556
bleu score for sentence  3  is  0.06108848866233461
bleu score for sentence  4  is  5.435893325369165e-155
bleu score for sentence  5  is  4.557665853237121e-155
bleu score for sentence  6  is  1.1640469867513693e-231
bleu score for sentence  7  is  3.7192222052000083e-155
bleu score for sentence  8  is  5.264471767154621e-155
bleu score for sentence  9  is  1.384292958842266e-231
bleu score for sentence  10  is  1.814573471947515e-78
bleu score for sentence  11  is  0.10148014680537136
bleu score for sentence  12  is  0.2025952259967125
bleu score for sentence  13  is  0.07429539148559132
bleu score for sentence  14  is  0.0943672664503419
bleu score for sentence  15  is  1.4864333923899637e-78
bleu score for sentence  16  is  2.3648538565211256e-78
bleu score for sentence  17  is  0.4113359224778876
bleu score for sentence  18  

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("amandyk/mt5-kazakh-english-translation")
tokenizer = AutoTokenizer.from_pretrained("amandyk/mt5-kazakh-english-translation")

translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="eng", tgt_lang="kaz", batch_size=64, device=device, max_length=400)


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
reference_sentences = [[tokenizer.tokenize(sentence)] for sentence in test_data['sentence_kaz_Cyrl']]

In [ ]:
total_bleu = 0.0
num_samples = len(test_data)
for i in range(num_samples):
  translated_sentence = translator(test_data['sentence_eng_Latn'][i])[0]['translation_text']
  translated_tokens = tokenizer.tokenize(translated_sentence)
  bleu_score = sentence_bleu(reference_sentences[i], translated_tokens)
  print("bleu score for sentence ", i, " is ", bleu_score)
  total_bleu += bleu_score

average_bleu = total_bleu / num_samples
print("Average bleu score is ", average_bleu)

bleu score for sentence  0  is  0.0939198349679984
bleu score for sentence  1  is  0.29961240360448776
bleu score for sentence  2  is  0.23636441127979887
bleu score for sentence  3  is  0.18883084687096116
bleu score for sentence  4  is  0.07916899456647894
bleu score for sentence  5  is  0.08217199674521214


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


bleu score for sentence  6  is  2.0038552016204957e-78
bleu score for sentence  7  is  1.4471931641118815e-78
bleu score for sentence  8  is  3.1872147730727704e-78


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


bleu score for sentence  9  is  1.1808001696991507e-231
bleu score for sentence  10  is  0.0829955458497328
bleu score for sentence  11  is  0.35658714053611246
bleu score for sentence  12  is  0.35427171284436915
bleu score for sentence  13  is  0.2394676514093617
bleu score for sentence  14  is  1.7078955065031708e-78
bleu score for sentence  15  is  0.18280028433117512
bleu score for sentence  16  is  0.1269383076503844
bleu score for sentence  17  is  0.34166820364217726
bleu score for sentence  18  is  0.2736877163680955
bleu score for sentence  19  is  0.1453721304259327
bleu score for sentence  20  is  0
bleu score for sentence  21  is  1.0832677820940877e-231
bleu score for sentence  22  is  2.564225419658708e-155
bleu score for sentence  23  is  0.23294902651565716
bleu score for sentence  24  is  0.16784459625186196
bleu score for sentence  25  is  0.309926477110924
bleu score for sentence  26  is  0.07443229846793785
bleu score for sentence  27  is  0.09521599948548468
bleu 

In [ ]:
!pip install googletrans

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 10.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15715 sha256=9aad66a78ce0f6808efe197884fcf8cf82c81cd7dedfb79b169b3c00d967aa53
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c